In [12]:
import numpy as np
import math
import shared
from mstsp import City, dist
import collections
import matplotlib.pyplot as plt
from typing import List
from nbc import NBC
import heapq
from itertools import chain

In [13]:
k=0
f=City(k)
print(f._get_maxfes(k))
print(f.city_num)
lb=0
ub=1
dim=f.city_num-1
a=np.random.uniform(lb, ub, size=dim)
print(a)
f._evaluate(a)
shared.i = 0
shared.MaxFES=f._get_maxfes(k)

60000.0
9
[0.43948324 0.4926471  0.25150058 0.96799079 0.28665057 0.78334317
 0.52488597 0.99257345]


In [14]:
temp=-1
num_clusters=-1
T=-1
M_factor=0
gen_mult=1

CR = 0.9  # MAGIC

NP = 150   ##150

MaxGens = 10

# if (num_clusters != -1):
#     # NP = min(5 + MaxGens // 2, max(10, 3*dim))
#     NP = int(num_clusters * max(10, 3*dim) * 2)  # let minsize be

#     # if (pop_hint != -1):
#     #     NP = max(math.ceil(shared.MaxFes/MaxGens),
#     #              int(pop_hint * max(10, 3*dim) * 2))


# g = 0

In [15]:
##all the functions

class TerminationCondition(Exception):
    def __init__(self, where):
        self.where = where

    def __str__(self):
        return self.where
    
class Agent():
    def __init__(self, x, f):
        # print(shared.i,shared.MaxFes)
        self.val = x
        self.stagnation_count = 0
        try:
            # TODO: actually, check bounds 
            self.t = f._evaluate(self.val)
        except:
            print("maybeHappened",x)
            self.t = float('-inf')
        

            # OR

            # self.val = np.maximum(x, lb)
            # self.val = np.minimum(x, ub)
            #and try again
    #    print(self.t)
            # print("math error:",x)   
        if shared.i > shared.MaxFES:
    #        print(shared.i)
            raise TerminationCondition("YOLO")
        else:
            shared.i += 1

        if hasattr(self.t, "__iter__"):
            self.fitness = self.t[0]
        else:
            self.fitness = self.t
        

    def __lt__(self, other):
        # inverse because of min heap
        return self.fitness >= other.fitness

def uniform_rand_init(lb, ub, dim, N, f) -> List[Agent]:
    return [Agent(np.random.uniform(lb, ub, size=dim), f) for _ in range(N)]

def permute_softmax3(edges, edge_lengths, temp=1):
    ans = []
    count = 0

    while edges:

        soft = softmax(edge_lengths*temp)
        num_indices = len(edges) - np.sum(soft == 0)

        indices = np.random.choice(
            len(edges), num_indices, replace=False, p=soft)

        for i in indices:
            ans.append(edges[i])
        for index in sorted(indices, reverse=True):
            del(edges[index])
        edge_lengths = np.delete(edge_lengths, indices)

        # print(count)
        # count += 1

    return ans

def NBC_minsize(population, minsize, temp=-1, phi=1.000,num_clusters=float("inf")):  # phi = 1   MAGIC numbers

    p = sorted(population)  # better to worse, reverse lt
    n = len(p) 
    if (n == 1):
        return [population]
    
    mem = {}

    def dist(i, j) -> float:
        if (i, j) in mem:
            return mem[(i, j)]
        if (j, i) in mem:
            return mem[(j, i)]
        x = p[i]
        y = p[j]
        t = np.sum(((x.val-y.val)**2))**.5
        mem[(j, i)] = t
        return t
    
    ans = {i: [] for i in range(0, n)} #[1:[],2:[],...]
    for i in range(1, n):
        t = min(range(0, i), key=lambda j: dist(i, j))
        ans[t].append(i)  #1:[i]
    
    parent = [-1]*n
    edges = []
    for i in ans:
        for j in ans[i]: # i is better, arrow points upwards,child up parent
            parent[j] = i
            edges.append((i, j))
     
    mu = sum(dist(p, c) for p, c in edges)/len(edges)
    fol = {}
    
    def follow(node):
        if node not in fol:
            if node in ans:
                fol[node] = 1 + sum(follow(i) for i in ans[node])
            else:
                fol[node] = 1
        return fol[node]
    
    if phi!=0:
        edges = [e for e in edges if dist(e[0], e[1]) > phi * mu] # adding counter ##if distance is > phi*mu
    edge_lengths = np.array([dist(e[0], e[1]) for e in edges])
    if (temp != -1):
        edges = permute_softmax3(edges, edge_lengths, temp=temp)
    else:
        edges.sort(key=lambda x: dist(x[0], x[1]), reverse=True)
    
    
    counter = 1
    count=0
    for e in edges:
        if(counter >= num_clusters):
            break
    
        if follow(e[1]) >= minsize:
            count+=1
            t = e[0]
            while parent[t] != -1:
                t = parent[t]
            if follow(t) - follow(e[1]) >= minsize:
                counter += 1
                ans[e[0]].remove(e[1])
                parent[e[1]] = - 1
                t = e[0]
                while parent[t] != -1:
                    fol[t] -= follow(e[1])
                    t = parent[t]

    def rec(node, homies):
        homies.append(node)
        if node in ans:
            for i in ans[node]:
                rec(i, homies)

    species = []
    for s in range(n):
        if parent[s] == -1:  # if i is a seed
            l = []
            rec(s, l)
            species.append(l)
    
    
#     for i in range(len(species)):
#         print(len(species[i]))
#     print(count)
    species = [[p[j] for j in i] for i in species]
    return species

  # TODO Magic numbers!!!

def balance(species):
    nums = [len(i) for i in species]
    rest = 0
    Lambda = 2  # Magic Numbers
    mu_avg = sum(nums)/len(nums)
    mu_lambda = round(Lambda * mu_avg)
    
    s = []
    for i in range(len(nums)):
        t = nums[i]
        if t > mu_lambda:
            rest += t - mu_lambda
            nums[i] = mu_lambda
        elif t < mu_avg:
            s.append(i)
    
    temp = int(rest/len(s))
    
    for i in s:
        nums[i] += temp

    if len(s) == 0:
        rest = 0
    else:
        rest %= len(s)

    i = 0
    while rest > 0:
        nums[s[i]] += 1
        i += 1
        rest -= 1

    return nums

def within_bounds(ans):
    if ((ans <= ub).all() and (ans >= lb).all()):
        return True
    else:
        return False

def rand_elem(l):
    return l[int(np.random.uniform() * len(l))]

def get_bounds(f):
    dim = f.get_dimension()
    ub = np.zeros(dim)
    lb = np.zeros(dim)
    # Get lower, upper bounds
    for k in range(dim):
        ub[k] = f.get_ubound(k)
        lb[k] = f.get_lbound(k)
    return ub, lb


def rand1(x, spe, F):
    r1 = rand_elem(spe).val
    r2 = rand_elem(spe).val
    r3 = rand_elem(spe).val
    # assert(((r1 + F * (r2-r3)) > np.array([0.25, 0.25])).all())
    return r1 + F * (r2-r3)

def rand2(x, spe, F):
    r1 = rand_elem(spe).val
    r2 = rand_elem(spe).val
    r3 = rand_elem(spe).val
    r4 = rand_elem(spe).val
    r5 = rand_elem(spe).val
    # assert(((r1 + F * (r2-r3) + F * (r4-r5)) > np.array([0.25, 0.25])).all())

    return r1 + F * (r2-r3) + F * (r4-r5)

def keypoint1(x, spe, F):
    r1 = rand_elem(NBC(spe)).val
    r2 = rand_elem(spe).val
    r3 = rand_elem(spe).val
    # assert(((r1 + F * (r2-r3)) > np.array([0.25, 0.25])).all())

    return r1 + F * (r2-r3)


def keypoint2(x, spe, F):
    r1 = rand_elem(NBC(spe)).val
    r2 = rand_elem(spe).val
    r3 = rand_elem(spe).val
    r4 = rand_elem(spe).val
    r5 = rand_elem(spe).val
    # assert(((r1 + F * (r2-r3) + F * (r4-r5)) > np.array([0.25, 0.25])).all())

    return r1 + F * (r2-r3) + F * (r4-r5)

def mutate(x, spe):
    type1 = np.random.uniform()
    type2 = np.random.uniform()
    # global i
    alpha = 0.5  # Magic numbers
    F1 = np.random.uniform(0.2, 0.8)  # Magic
    F2 = 0.5  # Magic
    per = 1 - (shared.i / shared.MaxFES)**alpha
    if type1 < per:
        if type2 < 0.5:
        #    print("1")
            v = rand1(x.val, spe, F1)
            while (not within_bounds(v)):
                v = rand1(x.val, spe, F1)
        else:
        #    print("2")
            v = rand2(x.val, spe, F2)
            while (not within_bounds(v)):
                v = rand2(x.val, spe, F2)
    
    else:
        if type2 < 0.5:
         #   print("3")
            v = keypoint1(x.val, spe, F1)
            while (not within_bounds(v)):
                v = keypoint1(x.val, spe, F1)
        else:
        #    print("4")
            v = keypoint2(x.val, spe, F2)
            while (not within_bounds(v)):
                v = keypoint2(x.val, spe, F2)

    # assert((v < ub).all())
    # assert((v > lb).all())

    return v

phi_g=1

def crossover(v, x, CR, f):
    j_rand = int(np.random.uniform() * len(x))
    t = 1 * (np.random.uniform(size=len(x)) <= CR)
    ans = v*t + (1-t)*x
    ans[j_rand] = v[j_rand]
    return Agent(ans, f)

def get_bounds(f):
    dim = f.city_num-1
    ub = np.zeros(dim)
    lb = np.zeros(dim)
    # Get lower, upper bounds
    for k in range(dim):
        ub[k] = 1
        lb[k] = 0
    return ub, lb

def select_best(specie):
    s = NBC_minsize(specie, 0, temp=-1, phi=phi_g)  # TODO MAGIC PARAM
    p = sum(len(i) for i in s)
    return max(s, key=lambda i: goodness(i, p))
# assert max(i.fitness for i in best) == best[0].fitness

def goodness(s, p):
    def avg_fitness(s):
        return sum(i.fitness for i in s)/len(s)
    bias = 2**0.5  # TODO good BIAS VALUE?
    f = avg_fitness(s)
    f += bias * np.sqrt(np.log(p/len(s)))
    return f

def generate(s, n):
    noise = np.random.normal(0, 0.1, size=(len(s[0].val), n))
    w = np.copy(s[0].val)
    w = w.reshape((w.shape[0], 1))
    t = noise + w
    l = lb.reshape((lb.shape[0], 1))
    u = ub.reshape((ub.shape[0], 1))

    t = np.maximum(t, l)
    t = np.minimum(t, u)
    t = t.T
    return [Agent(t[i], f) for i in range(n)]


def generate_MAB(specie, num):
    return generate(select_best(specie), num)

def generate_MAB_rec(specie, num, base_size=5, repeat_size=10000):
    if(len(specie) <= base_size or len(specie) >= (repeat_size-5)):
        return generate(select_best(specie), num)

    return generate_MAB_rec(select_best(specie), num, repeat_size=len(specie))

def generate_MAB_iter_rec(specie, num):
    copy = specie[:]
    for i in range(num):
        s = generate_MAB_rec(select_best(copy), 1)
        copy.extend(s)
    return copy[-num:]

def dist(x: Agent, y: Agent) -> float:
    return np.sum(((x.val-y.val)**2))**.5

def nearest(x, population, M):
    return heapq.nsmallest(M+1, population, key=lambda y: dist(x, y))

ub,lb=get_bounds(f)


def solve(population,f, lb, ub, dim,temp, M_factor=M_factor):
    CR = 0.9  # MAGIC
    T=30
    for g in range(MaxGens):
        genz=[]
        minsize = min(5+g//2, max(10, 3*dim))
        species = NBC_minsize(population, minsize, phi=0, temp=temp, num_clusters=float("inf"))
        species = [sorted(x) for x in species]
        avg_fit = sum(x[0].fitness for x in species) / len(species)
    #     print(avg_fit)
        print(g)

        nums = balance(species)

            # if (pop_hint != -1):
            #     NP = max(math.ceil(shared.MaxFes/MaxGens),
            #              int(pop_hint * max(10, 3*dim) * 2))
    #     g = 0
    #     try: 
        for s, n in zip(species, nums):
            for i in range(n):
                if i<len(s):
                    x=s[i]
                    v=mutate(x,s)
                    u = crossover(v, x.val, CR, f)
                    x.stagnation_count+=1
                    w = u if u.fitness < x.fitness else x
                    genz.append(w)
                else:
                    ws = generate_MAB_iter_rec(s, n-len(s))
                    genz.extend(ws)
                    break

        population = genz
#         for i in range(len(population)):
#             print(population[i].fitness)

#         archive = []
#         bestworse = []

#         if(T != -1):


#         population = {i: True for i in population}

            # M = int(4 + np.random.uniform() * (21-4))
            # M = sum(nums)/len(nums) # TODO use minsize?
#             M = (1-M_factor)*minsize + M_factor*sum(nums)/len(nums)

#             for x in population:
#                 if population[x] and x.stagnation_count >= T:
#                     # print("Archived")
#                     neighbours = nearest(x, population, math.floor(M))
#                     worse = [i for i in neighbours if i.fitness < x.fitness]
#                     worse.append(x)  # x is last

#                     bestworse.append(x)  # x is last

#                     for i in worse:
#                         population[i] = False
#                     archive.append(worse)
            # print(population)
  #          print(bestworse)
#             population = [i for i in population if population[i]]
            # print(population)
            # before = shared.i
#             population.extend(uniform_rand_init(lb, ub, dim, NP-len(population), f))
            # check_i(before, 0)

            # print("                         ", time.time()-old)
            # old=time.time()
#     except:
#     return np.array([i.val for i in chain(population, bestworse)])
    return population

In [16]:
prog_counter=0 ##number of variables in genz

num_count=10 ##number of times the program has run
archive = []
bestworse = []

In [17]:
population = uniform_rand_init(lb, ub, dim, NP, f)
# for i in range(num_count):
#     population = uniform_rand_init(lb, ub, dim, NP, f)
# g = 0
# while g<MaxGens:
#     minsize = min(5+g//2, max(10, 3*dim))
#     species = NBC_minsize(population, minsize, phi=0, temp=temp, num_clusters=float("inf"))
#     species = [sorted(x) for x in species]
#     avg_fit = sum(x[0].fitness for x in species) / len(species)
# #     print(avg_fit)
#     print(g)
    
#     nums = balance(species)
    
solu = solve(population,f, lb, ub, dim,temp, M_factor)
    
#     g = g+1
    

0
1
2
3
4
5
6
7
8
9


In [18]:
print(len(solu))

150


In [19]:
print(solu)

[<__main__.Agent object at 0x7fe5dda7bb80>, <__main__.Agent object at 0x7fe5dd979a90>, <__main__.Agent object at 0x7fe5dd9799d0>, <__main__.Agent object at 0x7fe5dda5af10>, <__main__.Agent object at 0x7fe60c1c5f70>, <__main__.Agent object at 0x7fe5dda7b040>, <__main__.Agent object at 0x7fe5ddaa0be0>, <__main__.Agent object at 0x7fe5dda7c370>, <__main__.Agent object at 0x7fe5dd9c8f10>, <__main__.Agent object at 0x7fe5dd979c10>, <__main__.Agent object at 0x7fe5dda84ca0>, <__main__.Agent object at 0x7fe5ddaae280>, <__main__.Agent object at 0x7fe5dda5aa00>, <__main__.Agent object at 0x7fe5ddaa0e20>, <__main__.Agent object at 0x7fe5dda7c340>, <__main__.Agent object at 0x7fe5dda7c760>, <__main__.Agent object at 0x7fe5dd9f2d90>, <__main__.Agent object at 0x7fe5dda5a1f0>, <__main__.Agent object at 0x7fe60c1bfcd0>, <__main__.Agent object at 0x7fe5dda7c970>, <__main__.Agent object at 0x7fe60c1c5730>, <__main__.Agent object at 0x7fe5ddaaeac0>, <__main__.Agent object at 0x7fe5dda7b2e0>, <__main__.

In [20]:
for i in range(len(solu)): 
    a = solu[i].fitness
    print(a)

1031.0
977.0
872.0
900.0
891.0
861.0
861.0
848.0
848.0
829.0
829.0
808.0
804.0
949.0
944.0
940.0
895.0
891.0
891.0
865.0
861.0
966.0
882.0
867.0
859.0
854.0
840.0
840.0
839.0
813.0
796.0
796.0
790.0
796.0
791.0
791.0
791.0
944.0
931.0
929.0
880.0
870.0
870.0
855.0
821.0
866.0
832.0
827.0
809.0
809.0
941.0
929.0
813.0
884.0
872.0
872.0
848.0
845.0
828.0
828.0
815.0
813.0
813.0
813.0
803.0
928.0
921.0
884.0
778.0
778.0
724.0
847.0
847.0
843.0
817.0
793.0
778.0
778.0
778.0
829.0
852.0
886.0
829.0
789.0
865.0
829.0
789.0
766.0
708.0
708.0
680.0
861.0
680.0
830.0
810.0
804.0
776.0
765.0
763.0
694.0
861.0
862.0
856.0
846.0
748.0
816.0
758.0
788.0
775.0
758.0
744.0
731.0
694.0
718.0
704.0
699.0
699.0
699.0
681.0
681.0
681.0
681.0
789.0
831.0
820.0
820.0
740.0
795.0
781.0
747.0
740.0
737.0
824.0
819.0
814.0
809.0
804.0
783.0
781.0
724.0
778.0
778.0
775.0
765.0
680.0
765.0
755.0
724.0
724.0
680.0


In [ ]:
# prog_count=len(genz)
# print(prog_count)
# kii=np.array([])
# for i in range(len(solu)): 
#     kii=np.append(kii,solu[i].fitness)
    

# kii=np.sort(kii)
# print(kii)

In [ ]:
dictAr = {}
for i in range(len(genz)): 
    genz[i].val
    a=genz[i].val
    ##print(a)
    __num=f.city_num
    pathh=np.empty((__num,2))
    pathh[0][0]=0
    pathh[0][1]=0
    for j in range(__num-1):
        pathh[j+1][0] = a[j]
        pathh[j+1][1] = int(j+1)
        pathh[j+1][1]=int(pathh[j+1][1])
    columnIndex=0
    pathh=pathh[pathh[:,columnIndex].argsort()]
    dictAr[genz[i].fitness] = pathh
dictAr = collections.OrderedDict(sorted(dictAr.items()))
excel = list(dictAr.keys())
print(excel[0])
print(dictAr[excel[0]])

In [ ]:
x,y=0,0
# class cardinate:
#     def __init__(self,a=0,b=0):
#         x=a
#         y=b
        
mstsp_ds=["simple1_9", "simple2_10", "simple3_10", "simple4_11", "simple5_12", "simple6_12",
        "geometry1_10", "geometry2_12", "geometry3_10", "geometry4_10", "geometry5_10", "geometry6_15",
        "composite1_28", "composite2_34", "composite3_22", "composite4_33", "composite5_35", "composite6_39",	
        "composite7_42", "composite8_45", "composite9_48",  "composite10_55", "composite11_59","composite12_60", "composite13_66"]
filename = "benchmark_MSTSP/" + str(mstsp_ds[k]) + ".tsp"
file_ = open(filename,"r")
# Reading from the file
content = file_.readlines()

# Iterating through the content
# Of the file
city_vec=np.empty((f.city_num,2),dtype=float)
vec_x=np.array([])
vec_y=np.array([])

lll=0
for line in content:
    x, y = list(map(int, line.split()))
    city_vec[lll][0]=x
    city_vec[lll][1]=y
    lll+=1
for i in range(f.city_num):
    kk=int(dictAr[excel[0]][i][1])
    print(kk)
    xx=city_vec[kk][0]
    yy=city_vec[kk][1]
    vec_x=np.append(vec_x,xx)
    vec_y=np.append(vec_y,yy)

vec_x=np.append(vec_x,city_vec[int(dictAr[excel[0]][0][1])][0])
vec_y=np.append(vec_y,city_vec[int(dictAr[excel[0]][0][1])][1])
print(vec_x)

In [ ]:
print(vec_x)
plt.plot(vec_x[:], vec_y[:])

plt.show()

In [ ]:
archive=np.append(archive,genz)

In [ ]:
print(archive)